In [1]:
from iresnet import iresnet50

In [3]:
import zz
zz.summary(iresnet50(),(3,112,112))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               Input          [-1, 3, 112, 112]               0
            Conv2d-1         [-1, 64, 112, 112]           1,728
       BatchNorm2d-2         [-1, 64, 112, 112]             128
             PReLU-3         [-1, 64, 112, 112]              64
       BatchNorm2d-4         [-1, 64, 112, 112]             128
            Conv2d-5         [-1, 64, 112, 112]          36,864
       BatchNorm2d-6         [-1, 64, 112, 112]             128
             PReLU-7         [-1, 64, 112, 112]              64
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
           Conv2d-10           [-1, 64, 56, 56]           4,096
      BatchNorm2d-11           [-1, 64, 56, 56]             128
      IBasicBlock-12           [-1, 64, 56, 56]               0
      BatchNorm2d-13           [-1, 64,

In [1]:
import torch
import zz
from torch import nn

All seed set to 42
selected device cuda:0


In [2]:
trainL, testL,_,_ = zz.get_cifar10_mlleaks_fast(batch_size=128)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.l1 = nn.Sequential(
            nn.Conv2d(3,32,5,padding=2), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1152,128),
            nn.Tanh(),
            nn.Linear(128,10)
        )
    def forward(self, image):
        return self.l1(image)

zz.summary(CNN(), (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               Input            [-1, 3, 32, 32]               0
            Conv2d-1           [-1, 32, 32, 32]           2,432
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 32, 12, 12]          25,632
       BatchNorm2d-6           [-1, 32, 12, 12]              64
              ReLU-7           [-1, 32, 12, 12]               0
         MaxPool2d-8             [-1, 32, 6, 6]               0
           Flatten-9                 [-1, 1152]               0
           Linear-10                  [-1, 128]         147,584
             Tanh-11                  [-1, 128]               0
           Linear-12                   [-1, 10]           1,290
              CNN-13                   

In [25]:
torch.load('zz.pth')

CNN(
  (l1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=1152, out_features=128, bias=True)
    (10): Tanh()
    (11): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [23]:
from tqdm import tqdm
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
net = CNN().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.001)
for i in range(70):
    net.train()
    loss_sum = 0
    for x,y in tqdm(trainL):            
        x,y=x.cuda(),y.cuda()
        optimizer.zero_grad()
        outputs = net(x)
        loss = criterion(outputs,y)
        loss.backward()
        loss_sum += loss
        optimizer.step()
    # print(loss_sum)
    net.eval()
    if i%5==0:
        zz.get_accuracy(net,trainL,testL)

83it [00:00, 415.93it/s]                        
83it [00:00, 415.45it/s]                        

Train acc:	0.51901
Test  acc:	0.48375



83it [00:00, 415.60it/s]                        
83it [00:00, 415.08it/s]                        
83it [00:00, 416.48it/s]                        
83it [00:00, 415.22it/s]                        
83it [00:00, 416.83it/s]                        
  0%|          | 0/82 [00:00<?, ?it/s]

Train acc:	0.70913
Test  acc:	0.59715


83it [00:00, 417.20it/s]                        
83it [00:00, 419.80it/s]                        
83it [00:00, 433.82it/s]                        
83it [00:00, 437.85it/s]                        
83it [00:00, 436.36it/s]                        
  0%|          | 0/82 [00:00<?, ?it/s]

...

83it [00:00, 425.18it/s]                        
83it [00:00, 422.61it/s]                        
83it [00:00, 427.99it/s]                        
83it [00:00, 424.91it/s]                        
83it [00:00, 427.58it/s]                        
  0%|          | 0/82 [00:00<?, ?it/s]

Train acc:	1.00000
Test  acc:	0.64639


83it [00:00, 425.89it/s]                        
83it [00:00, 427.16it/s]                        
83it [00:00, 427.35it/s]                        
83it [00:00, 428.18it/s]                        
83it [00:00, 426.49it/s]                        
  0%|          | 0/82 [00:00<?, ?it/s]

Train acc:	1.00000
Test  acc:	0.64629


83it [00:00, 428.12it/s]                        
83it [00:00, 438.48it/s]                        
83it [00:00, 448.09it/s]                        


In [24]:
torch.save(net,'zz.pth')